In [1]:
import math
import datetime
import pandas as pd
import numpy as np
import time
from statistics import median
from scipy.stats import iqr

from sklearn.preprocessing import OrdinalEncoder

import timeit
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
data_path = '../data/'

In [3]:
items = pd.read_csv('{}items.csv'.format(data_path), sep='|')
infos = pd.read_csv('{}infos.csv'.format(data_path), sep='|')
orders = pd.read_csv('{}orders.csv'.format(data_path), sep='|')

items_save = items.copy()
orders_save = orders.copy()
infos_save = infos.copy()

In [4]:
items= items_save.copy()

In [5]:
orders = orders_save.copy()

In [6]:
infos = infos_save.copy()

# Reformat Catagories

In [7]:
items["category2"] = items.apply(lambda row: f"{int(row['category1'])}-{int(row['category2'])}", axis=1)
items["category3"] = items.apply(lambda row: f"{row['category2']}-{int(row['category3'])}", axis=1)

enc = OrdinalEncoder()
fitted = enc.fit_transform(items[['category2', 'category3']]) + 1
items['category2'] = fitted[:,0]
items['category3'] = fitted[:,1]
items = items.astype({'category1': 'int32', 'category2': 'int32', 'category3': 'int32'})

In [8]:
items.head()

,itemID,brand,manufacturer,customerRating,category1,category2,category3,recommendedRetailPrice
0,1,0,1,4.38,1,1,1,8.84
1,2,0,2,3.00,1,2,2,16.92
2,3,0,3,5.00,1,3,3,15.89
3,4,0,2,4.44,1,2,2,40.17
4,5,0,2,2.33,1,1,1,17.04


# Extract additional features on the time-dimension

In [9]:
orderss=orders.head(5)
orderss

,time,transactID,itemID,order,salesPrice
0,2018-01-01 00:01:56,2278968,450,1,17.42
1,2018-01-01 00:01:56,2278968,83,1,5.19
2,2018-01-01 00:07:11,2255797,7851,2,20.47
3,2018-01-01 00:09:24,2278968,450,1,17.42
4,2018-01-01 00:09:24,2278968,83,1,5.19


In [10]:
start = time.clock()

orders['daytime'] = pd.to_datetime(orders["time"])

# Date
orders["date"] = pd.DatetimeIndex(orders['daytime']).date

# Month of the year
orders['month'] = pd.DatetimeIndex(orders['daytime']).month

# Day of the month
orders['day'] = pd.DatetimeIndex(orders['daytime']).day

# Weekday: Starts with 0 = Monday, 6 = Sunday
orders['weekday'] = pd.DatetimeIndex(orders['daytime']).weekday

# Just the date, no time
orders['time'] = pd.DatetimeIndex(orders['daytime']).time

# Hour of the day
orders['hour'] = pd.DatetimeIndex(orders['daytime']).hour

# Calendar week
orders['calendar_week'] = pd.DatetimeIndex(orders['daytime']).week


end = time.clock()
print(end - start)


2.6076289999999993


In [11]:
ex_items = items.copy()
ex_infos = infos.copy()
ex_orders = orders.copy()

# Feature generation: Price deviations

- Diff Sales vs Recommended Price

Item A: 60x für 30€, 30x für 10€

Features:
- Lowest Price
- Highest Price
- Mean Price
- Units Sold with lowest price
- Units Sold with highest price

In [12]:
joined = pd.merge(left=pd.merge(left=items, right=infos, on='itemID'), right=orders, on='itemID')

In [13]:
joined['orders_priceDifference'] = joined['salesPrice'] - joined['recommendedRetailPrice']
joined['orders_relPriceDifference'] = joined['orders_priceDifference'] / joined['recommendedRetailPrice']

In [14]:
ex_orders['priceDifference'] = joined['orders_priceDifference']
ex_orders['relPriceDifference'] = joined['orders_relPriceDifference']

In [15]:
grouped_orders = orders.groupby("itemID")

In [16]:
#Minimal Price of an Item
if 'minSalesPrices' not in list(ex_items.columns):
    minprices = grouped_orders.min()["salesPrice"].rename('minSalesPrice')
    ex_items = pd.merge(left=ex_items, right=minprices, on='itemID', how="left")

In [17]:
#Maximal Price of an Item
if 'maxSalesPrices' not in list(ex_items.columns):
    maxprices = grouped_orders.max()["salesPrice"].rename('maxSalesPrice')
    ex_items = pd.merge(left=ex_items, right=maxprices, on='itemID', how="left")

In [18]:
#Mean Price of an Item
if 'meanSalesPrices' not in list(ex_items.columns):
    meanprices = grouped_orders.mean()["salesPrice"].rename('meanSalesPrice')
    ex_items = pd.merge(left=ex_items, right=meanprices, on='itemID', how="left")

In [19]:
#ex_items.drop(["minSalesPriceCounts", "maxSalesPriceCounts"], inplace=True, axis=1)

In [20]:
#x = grouped_orders.group_by(salesPrice).sum()["Orders"]
counts = orders.groupby(["itemID", "salesPrice"]).sum()["order"]

#Sum of Orders Price at min price of an Item
if 'minSalesPriceCounts' not in list(ex_items.columns):
    counts = counts.rename("minSalesPriceCounts")
    ex_items = pd.merge(left=ex_items, right=counts, left_on=['itemID', 'minSalesPrice'], right_on=['itemID', 'salesPrice'], how="left")

#Sum of Orders Price at max price of an Item
if 'maxSalesPriceCounts' not in list(ex_items.columns):
    counts = counts.rename("maxSalesPriceCounts")
    ex_items = pd.merge(left=ex_items, right=counts, left_on=['itemID', 'maxSalesPrice'], right_on=['itemID', 'salesPrice'], how="left")

# Sales


- number of unites sold
- revenue
- % of Overall revenue

In [21]:
# number of unites sold
counts2 = orders.groupby("itemID").sum()["order"]
if 'unitsSold' not in list(ex_items.columns):
    counts2 = counts2.rename("unitsSold")
    ex_items = pd.merge(left=ex_items, right=counts2, on="itemID", how="left")

In [22]:
#revenue
if 'revenue' not in list(ex_items.columns):
    df = orders.groupby(["itemID", "salesPrice"]).sum()["order"].reset_index(level= [0,1])
    df["revenue"] = df["salesPrice"] * df["order"]
    #print(df.head())
    counts2 = df.groupby("itemID").sum()["revenue"].rename("revenue")
    ex_items = pd.merge(left=ex_items, right=counts2, on="itemID", how="left")
# of Overall revenue
    total=ex_items["revenue"].sum()
    ex_items["relRevenue"] = ex_items["revenue"] / total

# Frequency of Sales

- first Date of Sell
- Last Date of Sell

- Is an item sold weekly within its sales period? (with x jokers)
- Is an item sold dayly within its sales period?
- Is an item sold monthly within its sales period?

- recentlysold: days since last date of sale


In [23]:
#ex_items.drop(["datefirstSell", "datelastSell"], inplace=True, axis=1)

In [24]:
#First date of Sell
if 'dateFirstSell' not in list(ex_items.columns):
    date = grouped_orders.min()["date"].rename("dateFirstSell")
    ex_items = pd.merge(left=ex_items, right=date, on="itemID", how="left")

if 'dateLastSell' not in list(ex_items.columns):
    date = grouped_orders.max()["date"].rename("dateLastSell")
    ex_items = pd.merge(left=ex_items, right=date, on="itemID", how="left")

In [25]:
ex_orders.head()

,time,transactID,itemID,order,salesPrice,daytime,date,month,day,weekday,hour,calendar_week,priceDifference,relPriceDifference
0,00:01:56,2278968,450,1,17.42,2018-01-01 00:01:56,2018-01-01,1,1,0,0,1,-5.73,-0.64819
1,00:01:56,2278968,83,1,5.19,2018-01-01 00:01:56,2018-01-01,1,1,0,0,1,-5.73,-0.64819
2,00:07:11,2255797,7851,2,20.47,2018-01-01 00:07:11,2018-01-01,1,1,0,0,1,-5.73,-0.64819
3,00:09:24,2278968,450,1,17.42,2018-01-01 00:09:24,2018-01-01,1,1,0,0,1,-5.73,-0.64819
4,00:09:24,2278968,83,1,5.19,2018-01-01 00:09:24,2018-01-01,1,1,0,0,1,-5.73,-0.64819


In [36]:
def sold_daily(item):
    if isinstance(item["dateFirstSell"], float):
        return np.nan
    difference = (item['dateLastSell'] - item['dateFirstSell']).days + 1
    return ex_orders.loc[ex_orders["itemID"] == item["itemID"]]["date"].nunique() / difference

def sold_weekly (item):
    if isinstance(item["dateFirstSell"], float):
        return np.nan
    firstweek= item["dateFirstSell"].isocalendar()[1]
    lastweek = item["dateLastSell"].isocalendar()[1]
    diffweeks = lastweek - firstweek + 1
    return ex_orders.loc[ex_orders["itemID"] == item["itemID"]]["calendar_week"].nunique() / diffweeks

def sold_monthly (item):
    if isinstance(item["dateFirstSell"], float):
        return np.nan
    firstweek= item["dateFirstSell"].month
    lastweek = item["dateLastSell"].month
    diffmonths = lastweek - firstweek + 1
    return ex_orders.loc[ex_orders["itemID"] == item["itemID"]]["month"].nunique() / diffmonths

def sold_weekly_binary(item, joker = 0, threshhold = 3):
    if isinstance(item["dateFirstSell"], float):
        return False
    firstweek= item["dateFirstSell"].isocalendar()[1]
    lastweek = item["dateLastSell"].isocalendar()[1]
    diffweeks = lastweek - firstweek + 1 
    if diffweeks <= threshhold:
        return False
    return ex_orders.loc[ex_orders["itemID"] == item["itemID"]]["calendar_week"].nunique() == diffweeks + joker

In [29]:
ex_items["soldDaily"]  = ex_items.apply(lambda row: sold_daily(row), axis=1)

In [37]:
ex_items["soldWeekly"]  = ex_items.apply(lambda row: sold_weekly_binary(row), axis=1)

In [38]:
ex_items["soldWeeklyContinuous"]  = ex_items.apply(lambda row: sold_weekly(row), axis=1)

In [31]:
ex_items["soldMonthly"] = ex_items.apply(lambda row: sold_monthly(row), axis=1)

In [32]:
last_training_date = ex_orders['date'].max()
ex_items["recentlySold"] = ex_items.apply(lambda row: (last_training_date - row['dateLastSell']).days + 1 if not isinstance(row['dateLastSell'], float) else np.nan, axis=1)



In [39]:
ex_items.head()

,itemID,brand,manufacturer,customerRating,category1,category2,category3,recommendedRetailPrice,minSalesPrice,maxSalesPrice,meanSalesPrice,minSalesPriceCounts,maxSalesPriceCounts,unitsSold,revenue,relRevenue,dateFirstSell,dateLastSell,soldDaily,soldWeekly,soldMonthly,recentlySold,soldWeeklyContinuous
0,1,0,1,4.38,1,1,1,8.84,3.11,3.43,3.111661,690.0,3.0,693.0,2156.19,2.418927e-05,2018-01-23,2018-06-26,0.206452,False,1.000000,4.0,0.739130
1,2,0,2,3.00,1,2,2,16.92,9.15,9.15,9.150000,5.0,5.0,5.0,45.75,5.132475e-07,2018-02-24,2018-05-29,0.042105,False,0.500000,32.0,0.266667
2,3,0,3,5.00,1,3,3,15.89,9.89,14.04,12.733253,91.0,240.0,331.0,4269.59,4.789850e-05,2018-01-10,2018-06-29,0.087719,False,0.833333,1.0,0.400000
3,4,0,2,4.44,1,2,2,40.17,13.01,14.10,13.798895,53.0,145.0,198.0,2734.03,3.067178e-05,2018-01-18,2018-06-28,0.067901,False,0.833333,2.0,0.416667
4,5,0,2,2.33,1,1,1,17.04,7.48,7.84,7.735556,52.0,128.0,180.0,1392.48,1.562157e-05,2018-01-06,2018-06-23,0.082840,False,0.833333,7.0,0.400000


# Persisting Tables with extended Features

In [40]:
ex_orders.to_csv('{}ex_orders_all.csv'.format(data_path), index=False, sep='|')
ex_items.to_csv('{}ex_items_all.csv'.format(data_path), index=False, sep='|')
ex_infos.to_csv('{}ex_infos_all.csv'.format(data_path), index=False, sep='|')